# Programming the STM32 Nucleo board



#### Learning goals

* Get to know the CubeIDE programming environment
* Software Interrupt Basics

#### Introduction

This task provides a step-by-step guide to configure an STM32-F446RE nucleo board to feed through voltage levels sampled with an ADC to the MCU's the built-in DAC. While there are numerous ways this may be accomplished, the suggested solution in this guide makes use of an ADC in continuous mode and uses interrupt requests to process voltagelevels once they've been converted to a digital representation. Happily, the CubeIDE provieds a graphical tool for generating all the various configuration settings for ADCs and similar, so there is no need to modify the registers manually.

#### Support Litterature

* [User manual STM32 Nucleo-64 boards](https://www.st.com/content/ccc/resource/technical/document/user_manual/98/2e/fa/4b/e0/82/43/b7/DM00105823.pdf/files/DM00105823.pdf/jcr:content/translations/en.DM00105823.pdf)
* [HAL documentation](Documents/um1725-description-of-stm32f4-hal-and-lowlayer-drivers-stmicroelectronics.pdf)
* [Reference Manual stm32f446xx](Documents/dm00135183-stm32f446xx-advanced-arm-based-32-bit-mcus-stmicroelectronics.pdf)

## a) Tools installation

Follow the installation [instructions on the ST.com website](https://wiki.st.com/stm32mcu/wiki/STM32StepByStep:Step1_Tools_installation) to set up the IDE. <br>
Take note: our nucleo board is an F446RE, which means we need to install the **STM32F4** MCU software package with embedded examples.

When you're done, try setting up the [blinking LED example in the "Getting started with GPIO" page](https://wiki.st.com/stm32mcu/wiki/Getting_started_with_GPIO) to check that you can now program the STM32 MCU.

Using the blinking LED example as a starting point, we will now add som functionality to make use of one ADC and one DAC.

## b) Configuring the DAC
1. Go to "Pinout & Configuration", select **DAC** and enable **OUT1**. Make sure "Output Buffer" is set to "Enable". The configuration menu should look like [this](Figures/DAC_config.PNG).

## c) Setting up the ADC in interrupt mode

1. Got to "Clock Configuration" and set the **APB2** Prescaler to **16**. This will set the peripheral clock PCLK2 which controls ADC clock speed to 5.25 MHz.
2. Go to "Pinout & Configuration", select **ADC1** and enable **IN0**. Once this is done we can configure the ADC. Below is a list of which parameters we want to adjust:
    - Clock Prescaler: **PCLK2 divided by 8**
    - Resolution: **12 bits (15 ADC Clock Cycles)**
    - Data Alignment: **Right Alignment**
    - Continuous Conversion Mode: **Enabled**
    - Sampling Time: **112 Cycles**
    
   The "Parameter Settings" tab in the "Configuration" window should now look like [this](Figures/ADC_config.png).
3. Go to "NVIC Settings" and click to enable "ADC1, ADC2 and ADC3 interrupts", [like this](Figures/ADC_interrupt.png).

The ADC will now continuosly update it's output register every time a new ADC conversion is completed, as well as send an interrupt request. This interrupt request will trigger the execution of a specific function which we will make use of later.

## d) Adding user code to `main.c`
Click "build" (or save the project) to genrate a code template. Then, open the `main.c` file in the `Core/Src` directory. In this file you will see certain fields marked with comments such as `/* USER CODE BEGIN <category> */` and `/* USER CODE END <category> */`. To add your own code to the template, the code must be added somewhere between these comments, otherwise it will be erased if the code template is rebuilt from a modified device configuration at a later time.

We wish to make the following changes:
1. Add a global unsigned int variable to store current ADC value. Mark as `__IO` (aka. volatile) to safeguard against race conditions. 
```c
/* USER CODE BEGIN PV */
__IO uint32_t adc_value; 
/* USER CODE END PV */
```
2. Define a function which is executed upon an ADC interrupt request. <br>In layman's terms: *Whenever the ADC reports the completion of another A/D conversion, execute this code immediately.*
```c
/* USER CODE BEGIN PFP */
void HAL_ADC_ConvCpltCallback(ADC_HandleTypeDef* hadc){
	adc_value = HAL_ADC_GetValue(&hadc1); // Read converted sample value from the ADC and store in adc_value.
	HAL_DAC_SetValue(&hdac, DAC_CHANNEL_1, DAC_ALIGN_12B_R, adc_value); // Write the value in adc_value to DAC channel 1 using 12 bit resolution.
}
/* USER CODE END PFP */
```
    - Important note: interrupt procedures should generally be kept as short as possible, with very few operations.

3. Add code to start up the ADC conversion process, as well as the DAC before the loop in the `main()` function .
```c
  /* USER CODE BEGIN 2 */
  HAL_Delay(100);  // Insert a short delay between ADC/DAC initialisation and startup.
  HAL_ADC_Start_IT (&hadc1); // Start up the ADC conversion process with interrupts.
  HAL_DAC_Start(&hdac, DAC_CHANNEL_1); // Start up the DAC with channel 1 as output.
  /* USER CODE END 2 */
```

## e) Verify the program is working 

A simple test to check that the program is working properly, can be to run the program in the debugger tool with break points in the ADC interrupt. Then, connect the ADC input to either $0 \text{ V}$ or $3.3 \text{ V}$, and use the debugger to verify that the ADC value in the program is updated to the correct value. You can consult the [user manual](https://www.st.com/content/ccc/resource/technical/document/user_manual/98/2e/fa/4b/e0/82/43/b7/DM00105823.pdf/files/DM00105823.pdf/jcr:content/translations/en.DM00105823.pdf) to find out which pin is ADC channel IN0.